In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# KrigingSurrogate

The KrigingSurrogate implements a simple Kriging interpolation method based on Gaussian Processes for Machine Learning (GPML) by Rasmussen and Williams. In the default configuration, the surrogate computes the mean of the predicted value. KrigingSurrogate also has an option “eval_rmse”, which can be set to True to also compute the RMSE (root mean squared error).

Here is a simple example where a Kriging model is used to approximate the output of a sinusoidal component.

In [2]:
import numpy as np

import openmdao.api as om

prob = om.Problem()

sin_mm = om.MetaModelUnStructuredComp()
sin_mm.add_input('x', 2.1)
sin_mm.add_output('f_x', 0., surrogate=om.KrigingSurrogate())

prob.model.add_subsystem('sin_mm', sin_mm)

prob.setup()

# train the surrogate and check predicted value
sin_mm.options['train_x'] = np.linspace(0,10,20)
sin_mm.options['train_f_x'] = .5*np.sin(sin_mm.options['train_x'])

prob.set_val('sin_mm.x', 2.1)

prob.run_model()

print(prob.get_val('sin_mm.f_x'))

[0.4316104]


In [3]:
from openmdao.utils.assert_utils import assert_near_equal
assert_near_equal(prob.get_val('sin_mm.f_x'), .5*np.sin(prob.get_val('sin_mm.x')), 1e-4)

1.3242306103195001e-05

## KrigingSurrogate Options

All options can be passed in as arguments or set later by accessing the options dictionary.

In [4]:
om.show_options_table("openmdao.surrogate_models.kriging.KrigingSurrogate")

Option,Default,Acceptable Values,Acceptable Types,Description
eval_rmse,False,"[True, False]",['bool'],Flag indicating whether the Root Mean Squared Error (RMSE) should be computed. Set to False by default.
lapack_driver,gesvd,N/A,['str'],"Which lapack driver should be used for scipy's linalg.svd.Options are 'gesdd' which is faster but not as robust,or 'gesvd' which is slower but more reliable.'gesvd' is the default."
nugget,2.220446049250313e-15,N/A,N/A,"Nugget smoothing parameter for smoothing noisy data. Represents the variance of the input values. If nugget is an ndarray, it must be of the same length as the number of training points. Default: 10. * Machine Epsilon"
training_cache,N/A,N/A,['str'],"Cache the trained model to avoid repeating training and write it to the given file. If the specified file exists, it will be used to load the weights"


## KrigingSurrogate Constructor

The call signature for the `KrigingSurrogate` constructor is:
```{eval-rst}
    .. automethod:: openmdao.surrogate_models.kriging.KrigingSurrogate.__init__
        :noindex:
```

## KrigingSurrogate Option Examples

**eval_rmse**

By default, `KrigingSurrogate` only returns the mean of the predicted outputs. You can compute both the mean and the root mean square prediction error by setting the “eval_rmse” option to True. The most recent calculation of error is stored in the component’s metadata, and accessed as follows.

In [5]:
import numpy as np

import openmdao.api as om

prob = om.Problem()

sin_mm = om.MetaModelUnStructuredComp()
sin_mm.add_input('x', 2.1)
sin_mm.add_output('f_x', 0., surrogate=om.KrigingSurrogate(eval_rmse=True))

prob.model.add_subsystem('sin_mm', sin_mm)

prob.setup()

# train the surrogate and check predicted value
sin_mm.options['train_x'] = np.linspace(0,10,20)
sin_mm.options['train_f_x'] = .5*np.sin(sin_mm.options['train_x'])

prob.set_val('sin_mm.x', 2.1)

prob.run_model()

print("mean")
print(prob.get_val('sin_mm.f_x'))
print("std")
print(sin_mm._metadata('f_x')['rmse'][0, 0])

mean
[0.4316104]
std
2.113695895107792e-05


In [6]:
assert_near_equal(prob.get_val('sin_mm.f_x'), .5*np.sin(prob.get_val('sin_mm.x')), 1e-4)
assert_near_equal(sin_mm._metadata('f_x')['rmse'][0, 0], 0.0, 1e-4)

2.113695895107792e-05